# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233597356965                   -0.50    7.0         
  2   -7.250274499569       -1.78       -1.41    1.0   7.93ms
  3   -7.251207970924       -3.03       -2.09    2.0   9.26ms
  4   -7.251149707227   +   -4.23       -2.01    2.0   9.43ms
  5   -7.251332685432       -3.74       -2.73    1.0   7.65ms
  6   -7.251338181919       -5.26       -3.20    1.0   8.05ms
  7   -7.251338748543       -6.25       -3.85    1.0   7.68ms
  8   -7.251338796852       -7.32       -4.36    2.0   9.45ms
  9   -7.251338798472       -8.79       -4.90    2.0   9.37ms
 10   -7.251338798680       -9.68       -5.41    2.0   9.33ms
 11   -7.251338798701      -10.68       -5.79    2.0   10.5ms
 12   -7.251338798704      -11.66       -6.09    2.0   10.0ms
 13   -7.251338798704      -12.12       -6.76    1.0   8.70ms
 14   -7.251338798705      -13.11       -7.20    3.0   10.7ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04813339781960447
[ Info: Arnoldi iteration step 2: normres = 0.6255780726680251
[ Info: Arnoldi iteration step 3: normres = 0.8521583069706491
[ Info: Arnoldi iteration step 4: normres = 0.25583663598172224
[ Info: Arnoldi iteration step 5: normres = 0.4863776557281307
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.57e-02, 7.00e-02, 4.23e-01, 2.27e-01, 1.61e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3170640271580983
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.19e-03, 1.95e-01, 2.05e-01, 1.10e-01, 5.37e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09958653784347368
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.56e-04, 2.01e-02, 1.32e-02, 5.81e-02, 7.11e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11038553064801612
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.74e-